In [ ]:
import datetime as dt
import pandas as pd

from fastcounting import helper, views, store, files, system, asum
%alias_magic t timeit

In [ ]:
import redis
r = redis.Redis(**helper.Helper().rediscred, decode_responses=True)

In [ ]:
# read your cleaned journal data from file

In [ ]:
df, filename = files.main_etl('2017-13')

In [ ]:
# this takes around 5 mins

In [ ]:
store.main('2017-13')

In [ ]:
# diff

In [ ]:
asum.main_total_diff('2017-13')

In [ ]:
# datefilter

In [ ]:
start = dt.datetime(2017,2,2).timestamp()
end = dt.datetime(2017,8,3).timestamp()

views.main_datefilter(start, end)

In [ ]:
# push your account system to redis database

In [ ]:
system.accountsystem_toredis()

In [ ]:
# example for date and accountfilter which returns a set which is the intersections of those two

In [ ]:
def filter(start_date, end_date, start_account, end_account):
    date_filter = r.zrangebyscore('atomic:date', start_date, end_date)
    account_filter = r.zrangebyscore('account:atomic', start_account, end_account)
    atomics = set.intersection(set(date_filter), set(account_filter))
    return atomics

In [ ]:
start = dt.datetime(2017,2,2).timestamp()
end = dt.datetime(2017,4,3).timestamp()

atomics = filter(start, end, 0, 99999)

In [ ]:
len(atomics)

In [ ]:
result_list = []
for atomic in atomics:
    row = r.hgetall(f'atomicID:{atomic[0]}')
    account = int(float(row['accountID']))
    general = row['generalID']
    row.update(r.hgetall(f"generalID:{general}"))
    row.update(r.hgetall(f"accountsystem:{account}"))
    result_list.append(row)
    print(account)
    r.xadd(f'account:{account}', row)
    break

In [ ]:
r.xrevrange('account:1830')

In [ ]:
%%t -n1
m = r.smembers('view')
result_list = []
for row in m:
    result_list.append(eval(row))

In [ ]:
pd.DataFrame(result_list).T

In [ ]:
atomics = r.zrangebyscore('general:atomic', general, general)

In [ ]:
r.hgetall(f'atomicID:{atomics[0]}')

In [ ]:
r.hgetall(f'atomicID:{atomics[1]}')

In [ ]:
r.hgetall(*)